# Notebook E-tivity 1 CE4021

Student name: Mohammed Abdullah Al-Jaaidi

Student ID: 23326727

<hr style="border:2px solid gray"> </hr>

## Imports

In [1]:
#None

If you believe required imports are missing, please contact your moderator.

<hr style="border:2px solid gray"> </hr>

## Task

### Option 1: Symbolic calculation of the derivative

#### Requirement 1:
Write a python function to symbolically calculate the derivative of a polynomial with one variable. The input should be a polynomial and the output should also be a polynomial.

#### Overall suggested solution steps:
1- Accept the symbolic polynomial as a string argument to the function.
2- devide the polynomial to its individual terms and store the terms as items in a python list (We can split the terms using split method and  “+” or “-” signs as separators)
3- Loop through the list and calculate then save the derivative of each term in three steps:
    1- Split the term into coefficient and exponent parts.
    2- Apply the power rule to compute the derivative term.
    3- Append the derivative term to a python list.
4- Join the derivative terms to form the derivative polynomial.
5- Return the result.

#### Initial solution code:

In [ ]:
def derivative_of_polynomial(polynomial):
    # Split the polynomial into its individual terms
    # TODO: add spliting on '-'
    terms = polynomial.split('+')
    # Initialize an empty list to store the derivative terms
    derivative_terms = []
    # Loop through each term and compute its derivative
    for term in terms:
        # Split the term into coefficient and exponent
        parts = term.split('x^')
        if len(parts) == 2:
            coefficient = int(parts[0]) if parts[0] else 1
            exponent = int(parts[1])
            # Apply the power rule to compute the derivative term
            derivative_coefficient = coefficient * exponent
            if exponent > 1:
                derivative_exponent = exponent - 1
                derivative_term = f"{derivative_coefficient}x^{derivative_exponent}"
            else:
                derivative_term = str(derivative_coefficient)
            derivative_terms.append(derivative_term)
    # Join the terms to form the derivative polynomial
    derivative_polynomial = '+'.join(derivative_terms)    
    return derivative_polynomial

#### Requirement 2:
Write a python function to evaluate (i.e. get the numerical value) of a polynomial for a given value of its variable. The input should be a polynomial and a value (point at which to evaluate the polynomial). The output should be a scalar.

#### Overall suggested solution steps:
1- Accept the symbolic polynomial and the given value as a string argument to the function.
2- devide the polynomial to its individual terms and store the terms as items in a python list (We can split the terms using split method and  “+” or “-” signs as separators)
3- Loop through each term and evaluate it for the given value

#### Initial solution code:

<hr style="border:2px solid gray"> </hr>

## Reflection

Write your reflection in below cell. With reference to the Rubric for E-tivity 1:
- Provide an accurate description of your code with advantages and disadvantages of design choices.
- Compare your approach to alternative (peer) approaches.
- Clearly describe how you have used your peers' work/input and how this has affected your own understanding / insights.

If you have not used peer input, you may state this, but your submission history in Gitlab should clearly show this is the case.